---
---
---
# **Studienarbeit in der Vorlesung 'Applied Big Data Analyitics'**
# **Implementation: Percent Price Oscillator (PPO) anhand des Data-Warehouse-Models**
---
---
---
* ### **Erstellt:** Wintersemeseter 2019-2020
* ### **Dozent:** Prof. Dr. Sebastian Leuoth
* ### **Autor:** Jan Gaida
* ### **Email:** jan.gaida@hof-university.de
* ### **Git:** [trading_2019](https://github.com/sleuoth-hof/trading_2019), [JanGaida](https://github.com/JanGaida) 
---
---
### *Powered by:*
### ***Hochschule für Angewandte Wissenschaften Hof***
[![Logo: Hochschule Hof](https://www.uni-assist.de/fileadmin/_processed_/4/7/csm_hof-university_logo_308ee8b37b.jpg)](https://www.hof-university.de/)
---
---
---
*© 2019-2020 Jan Gaida, Prof. Dr. Sebastian Leuoth. All Rights Reserved.*







---
# Laufzeit-Parameter
---

*Datenbasis Parameter*
---
---





In [0]:
#@markdown > ---
#@markdown > **Datenursprung** *─ definiert die zuwählende Datenbasis*
#@markdown > --- ---
Datenframe_Von = "ABDA2019-Repository" #@param ["ABDA2019-Repository", "Zu wählender Pfad", "Beispiel: Intel (erfordert eine manuell zu erstellende 'content/intc.csv'-Datei)"] {allow-input: false}

#@markdown

#@markdown > ---
#@markdown > **Zu ladende Dateien** *─ nur benutzt wenn 'Datenframe_Von' auf 'ABDA2019-Repository' eingestellt ist*
#@markdown > --- ---
Filenamen_Zu_Laden = '*.csv'  #@param {type: "string"}

#@markdown

#@markdown > ---
#@markdown > **Zu ladender Pfad** *─ nur benutzt wenn 'Datenframe_Von' auf 'Zu wählender Pfad' eingestellt ist*
#@markdown > --- ---
Filepfad_Zu_Laden = '*.csv'  #@param {type: "string"}
#@markdown > ---
#@markdown > **Anzahl der Zeichen im Chartnamen** *─ nur benutzt wenn 'Datenframe_Von' auf 'Zu wählender Pfad' eingestellt ist*
#@markdown > --- ---
Filenamen_Stellen_Von_Filepfad = 0  #@param {type: "integer"}

*Anzeige Parameter*
---
---

In [0]:
#@markdown > ---
#@markdown > **Preview**
#@markdown > --- ---
Anzeige_Aktiviert = False #@param {type:"boolean"}

#@markdown > ---
#@markdown > **Maximal anzuzeigende Zeilen im Preview**
#@markdown > --- ---
Anzeige_Limit = 10  #@param {type: "slider", min: 1, max: 10000}

#@markdown > ---
#@markdown > **Anzuzeigende Ziffern nach der Kommastelle**
#@markdown > --- ---
Anzeige_Zahlen_Genaugikeit = 3  #@param {type: "slider", min: 1, max: 10}

#@markdown > ---
#@markdown > **Template der verwendeten Diagramme** *─ vgl. Plotly*
#@markdown > --- ---
Anzeige_Template = "plotly_dark" #@param ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none'] {allow-input: false}

#@markdown > ---
#@markdown > **Anzuzeigender Zeitraum** *─ keine Auswirkung auf den Export*
#@markdown > --- ---
Anzeige_Start_Tag = '2013-04-01' #@param {type:"date"}
Anzeige_Ende_Tag = '2019-10-01' #@param {type:"date"}

Anzeige_Zahlen_Genaugikeit_str = str(Anzeige_Zahlen_Genaugikeit)

*Export Parameter*
---
---

In [0]:
#@markdown > ---
#@markdown > **Name des Export-Root-Ordners** *─ vgl. 'content/Export_Mainordner_Pfad/Export_Subordner_Pfad/Export_Ordnername'*
#@markdown > --- ---
Export_Mainordner_Pfad = 'ABDA2019'  #@param {type: "string"}
#@markdown > ---
#@markdown > **Name des Unterordner des Export-Root-Ordners** *─ vgl. 'content/Export_Mainordner_Pfad/Export_Subordner_Pfad/Export_Ordnername'*
#@markdown > --- ---
Export_Subordner_Pfad = 'spark_warehouse'  #@param {type: "string"}
#@markdown > ---
#@markdown > **Ordnername in dem erstellte CSV-Datein geschrieben werden** *─ vgl. 'content/Export_Mainordner_Pfad/Export_Subordner_Pfad/Export_Ordnername'*
#@markdown > --- ---
Export_Ordnername = 'ppo_csv_export'  #@param {type: "string"}

#@markdown

#@markdown > ---
#@markdown > **Ob zum Export die Pandas-Api benutzt werden soll**
#@markdown > --- ---
Pandas_CSV_Export = False #@param {type:"boolean"}

#@markdown

#@markdown > ---
#@markdown > **Ob nach Fertigstellung die erstellten Daten heruntergeladen werden sollen**
#@markdown > --- ---
Download_Export_Ordner = True #@param {type:"boolean"}

#@markdown > ---
#@markdown > **Ob die Erstellten CSV-Part-Files zusammengeführt werden sollen**
#@markdown > --- ---
Merge_Export_Part = True #@param {type:"boolean"}

#@markdown > ---
#@markdown > **Ob die Erstellten CSV-Part-Files gelöscht werden sollen** *- erfordert 'Merge_Export_Part'*
#@markdown > --- ---
Delete_Export_Part = True #@param {type:"boolean"}

---
# Datenbank
---
*Urspung: [More than 400 Cryptocurrency-Chartdata](https://www.kaggle.com/tencars/392-crypto-currency-pairs-at-minute-resolution/version/2)*

In [40]:
# clonen
!git clone https://github.com/sleuoth/ABDA2019.git

# entpacken 'btcusd'
!unzip ABDA2019/testdaten/cryptominuteresolution/btcusd.csv.zip
!mv btcusd.csv ABDA2019/testdaten/cryptominuteresolution/btcusd.csv

# resultat
!ls ABDA2019/testdaten/cryptominuteresolution

fatal: destination path 'ABDA2019' already exists and is not an empty directory.
Archive:  ABDA2019/testdaten/cryptominuteresolution/btcusd.csv.zip
  inflating: btcusd.csv              
abseth.csv	cndbtc.csv  foausd.csv	mkrdai.csv  rdnusd.csv	utkbtc.csv
absusd.csv	cndeth.csv  fsnbtc.csv	mkreth.csv  repbtc.csv	utketh.csv
agibtc.csv	cndusd.csv  fsneth.csv	mkrusd.csv  repeth.csv	utkusd.csv
agieth.csv	cnneth.csv  fsnusd.csv	mlneth.csv  repusd.csv	utneth.csv
agiusd.csv	cnnusd.csv  fttusd.csv	mlnusd.csv  reqbtc.csv	utnusd.csv
aidbtc.csv	csxeth.csv  fttust.csv	mnabtc.csv  reqeth.csv	veebtc.csv
aideth.csv	csxusd.csv  funbtc.csv	mnaeth.csv  requsd.csv	veeeth.csv
aidusd.csv	ctxbtc.csv  funeth.csv	mnausd.csv  rifbtc.csv	veeusd.csv
aiobtc.csv	ctxeth.csv  funusd.csv	mtnbtc.csv  rifusd.csv	vetbtc.csv
aioeth.csv	ctxusd.csv  geneth.csv	mtneth.csv  rlcbtc.csv	veteth.csv
aiousd.csv	dadbtc.csv  genusd.csv	mtnusd.csv  rlceth.csv	vetusd.csv
algbtc.csv	dadeth.csv  gnoeth.csv	ncabtc.csv  rlcusd.csv	vldeth.cs

---
# Weitere Frameworkinstallation
---

In [41]:
# jdk
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# tree
!apt-get install tree

# spark-package
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

# findspark
!pip install findspark

# numpy
!pip install numpy

# timeseries library
!pip install ts ts-flint

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.7.0-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


---
# Imports
---

In [42]:
# Für Pfade und andere OS-Funktionalität
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

if 'COLAB_TPU_ADDR' not in os.environ:
  print(">> Colab verbunden mit einer TPU")
else:
  print(">> Colab nicht verbunden mit einer TPU")

# Spark und Spark-SQL
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import input_file_name, col, collect_list, concat_ws, udf
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.window import Window

# Initialisierung Spark und Spark-SQL
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Für einfache Weiterverarbeitung des Datenframes (z.B. Visualisierung mit Plotty)
import pandas as pd
from pandas import Timestamp

# Für einfaches Handeln von Arrays aus Pandas-Datenframes
import numpy
from numpy import array

# Visualisierung als Diagramm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Für Darstellung von Datenframes
from IPython.display import display, HTML

# Für File-Download
from google.colab import files

# Um Zip-Datein zu erstellen
import shutil

# Für den merge
from glob import glob

>> Colab verbunden mit einer TPU


---
# Anwendung PPO-Indikator
---

*PPO Implementation basierend auf: [Investopedia.com](https://www.investopedia.com/terms/p/ppo.asp#formula-and-calculation-for-ppo)*


*Metadaten-Aggregation*
---
---

In [0]:
#
# Intialen Datenframe bilden
#

if Datenframe_Von == "ABDA2019-Repository":
  df_spark = spark.read.csv('./ABDA2019/testdaten/cryptominuteresolution/' + Filenamen_Zu_Laden, inferSchema = True, header = True)
  if Filenamen_Zu_Laden == '*.csv':
    start = 57
  else:
    start = 59
  end = 6

elif Datenframe_Von == "Zu wählender Pfad":
  df_spark = spark.read.csv( Filepfad_Zu_Laden, inferSchema = True, header = True)
  start = Filepfad_Zu_Laden.count + 15 - Filenamen_Stellen_Von_Filepfad - 4
  end = Filenamen_Stellen_Von_Filepfad

elif Datenframe_Von == "Beispiel: Intel (erfordert eine manuell zu erstellende 'content/intc.csv'-Datei)":
  df_spark = spark.read.csv('./intc.csv', inferSchema = True, header = True)
  start = 17
  end = 4

df_spark = df_spark \
  .withColumn('filepath', input_file_name()) \
  .withColumn('filename', (input_file_name()[start:end]) ) \
  .withColumn('timestamp', df_spark['time']/1000) \
  .withColumn('date', (df_spark['time']/1000).cast('timestamp'))

if Anzeige_Aktiviert:
  print("Type:", type(df_spark), "\n")
  print("Schema: ", end = '')
  df_spark.printSchema()
  df_spark_total_count = df_spark.count()
  print("Available rows:", df_spark_total_count, "\n\nDaten-Preview:")
  #df_spark.show(Anzeige_Limit, False) # Durch Darstellung mittels PandasDataframe ersetzt/optimiert
  display(HTML(df_spark.toPandas().to_html(index = False, max_rows = Anzeige_Limit)))

# Erstellen der SQL-Tabelle 
df_spark.createOrReplaceTempView("base_data")

if Anzeige_Aktiviert:
  df_spark_informationView = spark.sql("SELECT filename, count(*), min(date), max(date)  FROM base_data group by filename")
  print("Geladene Daten:")
  #df_spark_informationView.show(Anzeige_Limit, False) # Durch Darstellung mittels PandasDataframe ersetzt/optimiert
  display(HTML(df_spark_informationView.toPandas().to_html(index = False, max_rows = Anzeige_Limit)))

*Tag-Filter*
---
---

In [44]:
#
# PRO-TAG-FILTER
#

base_data_dayfiltered = spark.sql(
  "SELECT d.* FROM base_data d " \
  " where  (d.date, filename) in (select max(m.date), filename from base_data m group by date_format(m.date, \"y-M-d\"), filename)" )

# Erstellen der SQL-Tabelle
base_data_dayfiltered.createOrReplaceTempView("base_data_dayfiltered")

base_data_dayfiltered_total_count = base_data_dayfiltered.count()

base_data_dayfiltered_informationView = spark.sql("SELECT filename, count(*), min(date), max(date)  FROM base_data_dayfiltered group by filename")

if Anzeige_Aktiviert:
  print("Available rows:", base_data_dayfiltered_total_count, "(lost",(df_spark_total_count-base_data_dayfiltered_total_count), "rows)", "\n\nVerfügbare Daten pro Tag:")
  #base_data_dayfiltered_informationView.show(Anzeige_Limit) # Durch Darstellung mittels PandasDataframe ersetzt/optimiert
  display(HTML(base_data_dayfiltered_informationView.toPandas().to_html(index = False, max_rows = Anzeige_Limit)))

KeyboardInterrupt: ignored

*EMA-UDF*
---
---
Berrechnet nach: [tradistats.com](https://tradistats.com/exponentieller-gleitender-durchschnitt/)

In [0]:
#
# EMA-UDF
#

# see https://tradistats.com/exponentieller-gleitender-durchschnitt/
def ema(ar):
    if len(ar) > 0:
       SF  = 2/ (len(ar)+1)
       SFi = 1 - SF
       my_ema = ar[0]
       for i in ar:
           my_ema = (i * SF) + (my_ema * SFi)
    return my_ema

ema_udf = udf(ema, DoubleType())

*Windows*
---
---

In [0]:
#
# EMA-WINDOWS
#
win26 =  Window \
    .partitionBy("filename") \
    .orderBy("date") \
    .rowsBetween(-25, 0)

win12 = Window \
    .partitionBy("filename") \
    .orderBy("date") \
    .rowsBetween(-11, 0)

win9 = Window \
    .partitionBy("filename") \
    .orderBy("date") \
    .rowsBetween(-8, 0)

*PPO- & Signal-Berechnung*
---
---

In [0]:
#
# EMA_26 && EMA_12 CALCULATION
#

base_data_tmp_ema_1 = base_data_dayfiltered \
  .withColumn('win26_close_list', collect_list('close').over(win26)) \
  .withColumn('win12_close_list', collect_list('close').over(win12))

base_data_tmp_ema_2 = base_data_tmp_ema_1.select(
  "*",
  ema_udf(base_data_tmp_ema_1["win26_close_list"]).alias("EMA26"),
  ema_udf(base_data_tmp_ema_1["win12_close_list"]).alias("EMA12")
)

base_data_tmp_ema_2.createOrReplaceTempView("base_data_tmp_ema_2")

if Anzeige_Aktiviert:
  base_data_tmp_ema_2.show(Anzeige_Limit, False)

In [0]:
#
# PPO BERECHNUNG
#

base_data_tmp_ema_3 = spark.sql(
  " select *, (((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE)) " \
  " FROM base_data_tmp_ema_2 " \
  " order by filename asc, date desc "
)

#
# SIGNAL BERECHNUNG
#
base_data_tmp_ema_4 = base_data_tmp_ema_3 \
  .withColumn('win9_ema12sub26_list', collect_list('(((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))').over(win9))


base_data_tmp_ema_5 = base_data_tmp_ema_4.select(
  "*",
  base_data_tmp_ema_4['(((EMA12 - EMA26) / EMA26) * CAST(100 AS DOUBLE))'].alias("PPO"),
  ema_udf(base_data_tmp_ema_4["win9_ema12sub26_list"]).alias("SIGNAL")
)

base_data_tmp_ema_5.createOrReplaceTempView("base_data_tmp_ema_5")

if Anzeige_Aktiviert:
  base_data_tmp_ema_5.show(Anzeige_Limit, False)

*PPO_Histogram-Berechnung*
---
---

In [0]:
#
# PPO-HISTOGRAM BERECHNUNG
#

base_data_tmp_ema_6 = spark.sql(
  " select *, (PPO - SIGNAL)   " \
  " FROM base_data_tmp_ema_5 " \
  " order by filename asc, date desc "
)

base_data_ema = base_data_tmp_ema_6.select(
  "*",
  base_data_tmp_ema_6['(PPO - SIGNAL)'].alias("PPO_HISTOGRAM")
)

base_data_ema.createOrReplaceTempView("base_data_ema")

if Anzeige_Aktiviert:
  base_data_ema.show(Anzeige_Limit, False)

---
# Analyse 
---
*PPO-Analyse ebenfalls basierend auf: [Investopedia.com](https://www.investopedia.com/terms/p/ppo.asp#what-the-indicator-tells-you)*


*Windows*
---
---

In [0]:
#
# Analyse-Windows
#

win2 =  Window \
    .partitionBy("filename") \
    .orderBy("date") \
    .rowsBetween(-1, 0)

win3 = Window \
    .partitionBy("filename")\
    .orderBy("date") \
    .rowsBetween(-2, 0)

*Funktionen*
---
---
**→ ppo_trend_analysis** :

Auf den 'PPO' angewendet (window mind. 1):

> *When the PPO is above zero, that helps confirm an uptrend since the short-term EMA is above the longer-term EMA. When the PPO is below zero, the short-term EMA is below the longer-term EMA, which is an indication of a downtrend.*


**→ ppo_crossover_signal_analysis** :

Auf den 'PPO_HISTOGRAM' angewendet (window mind. 2):
> *The indicator generates a buy signal when the PPO line crosses above the signal line from below, and a sell signal occurs when the PPO line crosses  below the signal from above.*

Auf den 'PPO' angewendet (window mind. 2):
> *Centerline crossovers also generate trading signals. Traders consider a move from below to above the centerline as bullish, and a move from above to below the centerline as bearish.*

**→ ppo_technical_divergence_analysis:**

Auf den 'close' und 'PPO' (window mind. 2; ggf. 'close' mit 'open' tauschen):
> *Traders can also use the PPO to look for technical divergence between the indicator and price. For example, if the price of an asset makes a higher high, but the indicator makes a lower high, it may indicate the upward momentum is subsiding. Conversely, if an asset's price makes a lower low, but the indicator makes a higher low, it could suggest that the bears are losing their traction and the price could head higher soon.*


In [0]:
#
# Analyse-UDF
#

def ppo_trend_analysis(ar):
    output = ""
    count = len(ar)
    if count > 0:
      # Einzelabgleich
      if count == 1:
        current = ar[0]
        if current > 0:
          output = "▲ Aufwärtstrend" # "Aufwärtstrend"
        elif current < 0:
          output = "▼ Abwärtstrend" # "Abwärtstrend"

      # Vergleich mit den davorherigenden; ggf. mit mehreren vorherigen
      elif count > 1:
        last_position = count - 1
        next_to_last_position = last_position - 1

        last = ar[last_position] 
        next_to_last = ar[next_to_last_position]

        if last > 0 and next_to_last > 0:
          output = "▲ Aufwärtstrend"
        elif last < 0 and next_to_last > 0:
          output = "↘ neuer Abwärtstrend"
        elif last < 0 and next_to_last < 0:
          output = "▼ Abwärtstrend"
        elif last > 0 and next_to_last < 0:
          output = "↗ neuer Aufwärtstrend"

    return output

def ppo_crossover_signal_analysis(ar):
    output = ""
    count = len(ar)
    # Vergleich des letzten Crossovers
    if count > 1:
        last_position = count - 1
        next_to_last_position = last_position - 1
        
        last = ar[last_position] 
        next_to_last = ar[next_to_last_position]

        if last <= 0 and next_to_last > 0:
          output = "Verkauf-Signal"
        elif last >= 0 and next_to_last < 0:
          output = "Kauf-Signal"

    return output

def ppo_technical_divergence_analysis(close_ar, ppo_ar):
    output = ""
    count_close = len(close_ar)
    count_ppo = len(ppo_ar)
    
    # Arrays müssen selbe Größe haben
    if count_close == count_ppo and count_close > 1:
      # delta berechnen
      pointer = count_close - 1
      delta_close = close_ar[pointer]
      delta_ppo = ppo_ar[pointer]
      pointer -= 1

      while pointer >= 0:
        delta_close -= close_ar[pointer]
        delta_ppo -= ppo_ar[pointer]
        pointer -= 1

      # steigung berechnen
      div_close = delta_close / count_close
      div_ppo = delta_ppo / count_ppo

      if div_close > div_ppo:
        output = "↧ Abflachend"
      else:
        output = "↥ Wachsend"

    return output

ppo_trend_analysis_udf = udf(ppo_trend_analysis, StringType())
ppo_crossover_signal_analysis_udf = udf(ppo_crossover_signal_analysis, StringType())
ppo_technical_divergence_analysis_udf = udf(ppo_technical_divergence_analysis, StringType())

*Aggregation*
---
---

In [0]:
#
# SIGNAL ANALYSIS
#

base_data_ema_tmp_eval = base_data_ema \
  .withColumn('win2_ppo_list', collect_list('PPO').over(win2)) \
  .withColumn('win2_ppoh_list', collect_list('PPO_Histogram').over(win2)) \
  .withColumn('win2_close_list', collect_list('close').over(win2))

base_data_ema_eval_result = base_data_ema_tmp_eval.select(
  "*",
  ppo_trend_analysis_udf(base_data_ema_tmp_eval["win2_ppo_list"]).alias("TREND"),
  ppo_crossover_signal_analysis_udf(base_data_ema_tmp_eval["win2_ppo_list"]).alias("WEAK_SIGNAL"),
  ppo_crossover_signal_analysis_udf(base_data_ema_tmp_eval["win2_ppoh_list"]).alias("STRONG_SIGNAL"),
  ppo_technical_divergence_analysis_udf(base_data_ema_tmp_eval["win2_close_list"], base_data_ema_tmp_eval["win2_ppo_list"]).alias("DIVERGENCE")
)

base_data_ema_eval_result.createOrReplaceTempView("base_data_ema_eval_result")

if Anzeige_Aktiviert:
  base_data_ema_eval_result.show(Anzeige_Limit, False)

---
# Auswertung
---

*Für alle finalen Empfehlungen gilt eine Reichteweite von 0 (schwach) bis 3 (stark)*

**Halten**
---
---
*   **H0:** 
  * ▼ Abwärtstrend && ↧ Abflachend
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **H1:**
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
*   **H2:**
  * ▲ Aufwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
*   **H3:**
  * ▲ Aufwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↥ Wachsend

In [0]:
#
# Halten-SQL-Statement
#

holdingCaseStatement = "CASE WHEN (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"H1\"" +"\n" \
"WHEN (TREND = \"▲ Aufwärtstrend\" = \"↧ Abflachend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"H2\"" +"\n" \
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") OR (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"H3\"" +"\n" \
"ELSE \"H0\" END" +"\n"

**Kaufen**
---
---
*   **K0:** 
  * ▼ Abwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **K1:**
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
*   **K2:**
  * ▲ Aufwärtstrend && ↧ Abflachend
  * ↗ neuer Aufwärtstrend && ↥ Wachsend
*   **K3:**
  * ▲ Aufwärtstrend && ↥ Wachsend

In [0]:
#
# Kaufen-SQL-Statement
#

buyingCaseStatement = "CASE WHEN (TREND = \"▼ Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"K0\"" +"\n" \
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") OR (TREND = \"↗ neuer Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"K2\"" +"\n" \
"WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"K3\"" +"\n" \
"ELSE \"K1\" END" +"\n"

**Verkaufen**
---
---
*   **V0:** 
  * ▲ Aufwärtstrend && ↥ Wachsend
  * ▲ Aufwärtstrend && ↧ Abflachend
*   **V1:**
  * ↗ neuer Aufwärtstrend && ↥ Wachsend
  * ↘ neuer Abwärtstrend && ↥ Wachsend
*   **V2:**
  * ▼ Abwärtstrend && ↥ Wachsend
  * ↗ neuer Aufwärtstrend && ↧ Abflachend
  * ↘ neuer Abwärtstrend && ↧ Abflachend
*   **V3:**
  * ▼ Abwärtstrend && ↧ Abflachend

In [0]:
#
# Verkaufen-SQL-Statement
#

sellingCaseStatement = "CASE WHEN (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\")OR (TREND = \"▲ Aufwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"V0\"" +"\n" \
"WHEN (TREND = \"↗ neuer Aufwärtstrend\" = \"↥ Wachsend\" AND DIVERGENCE = \"↥ Wachsend\") OR (TREND = \"↘ neuer Abwärtstrend\" AND DIVERGENCE = \"↥ Wachsend\") THEN \"V1\"" +"\n" \
"WHEN (TREND = \"▼ Abwärtstrend\" AND DIVERGENCE = \"↧ Abflachend\") THEN \"V3\" " +"\n" \
"ELSE \"V2\" END" +"\n"

**Aggregation**
---
---
*vgl. dazu Investopedia-Link (s.h. Sections-Header)*

* Wenn beide Signale null sind ⇒ *halten*
* Ansonsten 'Strong-' > 'Weaksignal', für jeweils beide gilt:
  * KAUF-Signal ⇒ *kaufen*
  * VERKAUF-Signal ⇒ *verkaufen*

```
+----------------+----------------+------------+
|  WEAK_SIGNAL   | STRONG_SIGNAL  | EMPFEHLUNG |
+----------------+----------------+------------+
| null           | null           | HALTEN     |
| Kauf-Signal    | null           | KAUFEN     |
| Kauf-Signal    | Kauf-Signal    | KAUFEN     |
| ...            | ...            | ...        |
| Verkauf-Signal | null           | VERKAUFEN  |
| Verkauf-Signal | Verkauf-Signal | VERKAUFEN  |
| ...            | ...            | ...        |
| Kauf-Signal    | Verkauf-Signal | VERKAUFEN  |
| Verkauf-Signal | Kauf-Signal    | KAUFEN     |
+----------------+----------------+------------+
```


In [0]:
#
# Combiniertes-SQL-Statement
#

final_result_statement = "SELECT *," \
" CASE" \
"   WHEN (WEAK_SIGNAL = \"\" AND STRONG_SIGNAL = \"\") THEN " + holdingCaseStatement + \
"   WHEN (STRONG_SIGNAL = \"\") THEN " \
"     CASE" \
"        WHEN (WEAK_SIGNAL = \"Kauf-Signal\") THEN " + buyingCaseStatement + \
"        ELSE " + sellingCaseStatement + \
"     END" \
"   ELSE" \
"     CASE" \
"        WHEN (STRONG_SIGNAL = \"Kauf-Signal\") THEN " + buyingCaseStatement + \
"        ELSE " + sellingCaseStatement + \
"     END"  \
" END AS PPO_RESULT" \
" FROM base_data_ema_eval_result"

data_final_eval_result = spark.sql(final_result_statement)
data_final_eval_result.createOrReplaceTempView("data_final_eval_result")

---
# Visualisierung
---

*Parameter*
---

In [0]:
#
# Minimales bis Maximale Datum der geladenen Kurse
#

if Anzeige_Aktiviert:
  #base_data_dayfiltered_informationView.show(Anzeige_Limit) # Durch Darstellung mittels PandasDataframe ersetzt/optimiert
  display(HTML(base_data_dayfiltered_informationView.toPandas().to_html(index = False, max_rows = Anzeige_Limit)))

*Vorbereitung*
---

In [0]:
#
# Datenvorbereitung
#
if Anzeige_Aktiviert:
  # Daten von Auswertung in Dataframe auswählen
  data_eval_result = spark.sql(
    " select filename, Date(date), format_number(high, "+Anzeige_Zahlen_Genaugikeit_str+") high, format_number(low, "+Anzeige_Zahlen_Genaugikeit_str+") low, format_number(open, "+Anzeige_Zahlen_Genaugikeit_str+") open, format_number(close, "+Anzeige_Zahlen_Genaugikeit_str+") close" \
    " , format_number(PPO, "+Anzeige_Zahlen_Genaugikeit_str+") PPO, format_number(SIGNAL, "+Anzeige_Zahlen_Genaugikeit_str+") SIGNAL, format_number(PPO_HISTOGRAM, "+Anzeige_Zahlen_Genaugikeit_str+") PPO_HISTOGRAM " \
    " , TREND, WEAK_SIGNAL, STRONG_SIGNAL, DIVERGENCE, PPO_RESULT "\
    " FROM data_final_eval_result " \
    " WHERE date >= '"+ Anzeige_Start_Tag.strip() +" 00:00:00' AND date <= '"+ Anzeige_Ende_Tag.strip() +"23:59:59' " \
    " order by filename asc, date asc " 
  )

  # Vorbereiten der for-each-loop
  data_eval_filenames = spark.sql("select distinct filename from data_final_eval_result")
  data_eval_filenames_ar = [row.filename for row in data_eval_filenames.collect()]

else:
  print("Aufbereitung der Daten zur Visualisierung durch Parameter 'Anzeige_Aktiviert' übersprungen.")

Output
---

In [0]:
#
# Charts & Table
#

if Anzeige_Aktiviert:
  
  # Zusätzliches CSS in der Finalen Darstellung:
  hideInjectedFilenameColumn_CSS = "<style>" \
    "table td:nth-child(1) { display:none;}" \
    "table.dataframe thead th:first-child {display: none;}" \
    "</style>"

  # Darstellungs-Loop für alle Filenames:
  for currentFilename in data_eval_filenames_ar:

    # Gefilteter Dataframe und dessen Collection:
    current_data_df = data_eval_result.where(data_eval_result['filename'] == currentFilename) # alle Columns ausgewählt für informativere Dataframe-Darstellung
    current_data_collection = current_data_df.collect()

    # Aufbau der Charts:
    current_data_df_dates = [Timestamp(row.date) for row in current_data_collection]

    current_stock_chart = go.Figure(
      data = [ go.Candlestick(
          x = current_data_df_dates,
          open = [row.open for row in current_data_collection],
          high = [row.high for row in current_data_collection],
          low = [row.low for row in current_data_collection],
          close = [row.close for row in current_data_collection]
      )]
    )

    current_ppo_chart = go.Figure(
        data = [ go.Scatter(
            x = current_data_df_dates,
            y = [row.PPO for row in current_data_collection],
            mode = 'lines',
            name = 'PPO'
        ), go.Scatter(
            x = current_data_df_dates,
            y = [row.SIGNAL for row in current_data_collection],
            mode = 'lines',
            name = 'Signal'
        ), go.Bar(
            x = current_data_df_dates,
            y = [row.PPO_HISTOGRAM for row in current_data_collection],
            name = 'PPO-Histogram'
        )]
    )

    # Chart Styling:
    current_stock_chart.update_layout(
        title='Chart für CSV-Datei \'' + currentFilename + '\'',
        yaxis_title = 'Punkte',
        template = Anzeige_Template,
        xaxis_rangeslider_visible = False
    )

    current_ppo_chart.update_layout(
        title='PPO-Indikator für CSV-Datei \'' + currentFilename + '\'',
        yaxis_title = 'Änderung in %',
        template = Anzeige_Template
    )

    # Ausgabe:
    print("\n\nFilename: " + currentFilename + "\nZeitraum: " + Anzeige_Start_Tag + " → " + Anzeige_Ende_Tag + "\n")
    current_stock_chart.show()
    current_ppo_chart.show()
    print()
    #current_data_df.show() # Durch Darstellung mittels PandasDataframe ersetzt/optimiert
    display(HTML(current_data_df.toPandas().to_html(index = False, max_rows = Anzeige_Limit) + hideInjectedFilenameColumn_CSS))
    print()

else:
  print("Visualisierung durch Parameter 'Anzeige_Aktiviert' übersprungen.")

----
# CSV-Export
----
*Die Daten sollen nach Tage sotiert ausgegeben werden (vgl. Besprechung 13.12.19)*

*Darstellung des Datums nach [ISO-8601-Norm](https://lmgtfy.com/?q=ISO-8601) (z.B.: 2019-09-07)*

Ordner-Struktur
---

**aktueller Pfad sollte '~/content' sein**

In [0]:
#
# (Neuen) Ordner Erstellen
#

# Alten Ordner löschen:
if os.path.exists(os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername)):
  shutil.rmtree(os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername))

# Ordner-Checks (ggf. Erstellen der Ordner):
if not os.path.exists(Export_Mainordner_Pfad):
  os.mkdir(Export_Mainordner_Pfad)

if not os.path.exists(os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad)):
  os.mkdir(os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad))

if not os.path.exists(os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername)):
  os.mkdir(os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername))

Vorbereitung
---

In [0]:
#
# Outputt Daten sammeln
#

# Zu speicherende Daten als Dataframe auswählen:
data_output = spark.sql(
  " select filename, Date(date) date, open, close, high, low, volume, PPO_RESULT" \
  " FROM data_final_eval_result " \
  " order by filename asc, date asc " 
)
data_output.collect();

# Zu filternde Tage:
data_output_days = spark.sql("select distinct Date(date) date FROM data_final_eval_result")
data_output_days_ar = array([Timestamp(row.date) for row in data_output_days.collect()]) # Für die Ausgabe als Numpy-Array

Export
---

In [47]:
#
# Daten als csv exportieren
#

if Anzeige_Aktiviert:
  print("Zu exportierende Tage (Anzahl = " + str(data_output_days_ar.size) + "):\n")
  numpy.set_printoptions(threshold = Anzeige_Limit)
  print(data_output_days_ar)
  print("\nZu exportierender Datensatz:")
  display(HTML(data_output.toPandas().to_html(index = False, max_rows = Anzeige_Limit)))

for currentDay in data_output_days_ar:
    # Datenframe für den aktuellen Tag:
    currentOutput = data_output.where(data_output.date == currentDay)

    # Anpassung des Monats zu 2 Stellen
    if currentDay.month > 9:
      formattedMonth = str(currentDay.month)
    else:
      formattedMonth = '0' + str(currentDay.month)

    # Anpassung des Tages zu 2 Stellen
    if currentDay.day > 9:
      formattedDay = str(currentDay.day)
    else:
      formattedDay = '0' + str(currentDay.day)

    # Filename & Filepfad
    currentFilename = str(currentDay.year) + "-" + formattedMonth + "-" + formattedDay
    currentPath = os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername, currentFilename)
    destinationPath = os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername, (currentFilename + ".csv") )

    # Schreiben
    if Pandas_CSV_Export:
      currentOutput.toPandas().to_csv(currentPath, encoding = 'utf-8', index = False, mode = 'w')
    else:
      currentOutput.write.csv(currentPath, mode = 'overwrite', encoding = 'utf-8', header = True )

if Anzeige_Aktiviert:
  print("\n\nExport abgeschlossen, erstellte Dateien:")
  os.listdir(os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername))
else:
  print("\n\nExport abgeschlossen.")




Export abgeschlossen.


In [0]:


filename = "main.csv"

with open(filename, 'a') as singleFile:
    first_csv = True
    for csv in glob('ABDA2019/spark_warehouse/ppo_csv_export/2019-06-30/*.csv'):
        if csv == filename:
            pass
        else:
            header = True
            for line in open(csv, 'r'):
                if first_csv and header:
                    singleFile.write(line)
                    first_csv = False
                    header = False
                elif header:
                    header = False
                else:
                    singleFile.write(line)
    singleFile.close()

In [0]:
if Merge_Export_Part:
  for currentDay in data_output_days_ar:
    # Anpassung des Monats zu 2 Stellen
    if currentDay.month > 9:
      formattedMonth = str(currentDay.month)
    else:
      formattedMonth = '0' + str(currentDay.month)

    # Anpassung des Tages zu 2 Stellen
    if currentDay.day > 9:
      formattedDay = str(currentDay.day)
    else:
      formattedDay = '0' + str(currentDay.day)

    # Filename & Filepfad
    currentFilename = str(currentDay.year) + "-" + formattedMonth + "-" + formattedDay
    currentPath = os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername, currentFilename)
    exportFilename = os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername, (currentFilename + ".csv"))


    # Mergen | Credits: https://stackoverflow.com/a/59108574
    with open(exportFilename, 'a') as singleFile:
      first_csv = True

      for csv in glob((currentPath + '/*.csv')):
          if csv == filename:
              pass
          else:
              header = True

              for line in open(csv, 'r'):
                  if first_csv and header:
                      singleFile.write(line)
                      first_csv = False
                      header = False
                  elif header:
                      header = False
                  else:
                      singleFile.write(line)

      singleFile.close()

    #
    # Part-Ordner ggf. löschen
    #
    if Delete_Export_Part:
      if os.path.exists(currentPath):
        shutil.rmtree(currentPath)

In [0]:
#
# Daten ggf. als Download anbieten
#

if Download_Export_Ordner:
  shutil.make_archive('Exportdaten_PPO', 'zip', os.path.join(Export_Mainordner_Pfad, Export_Subordner_Pfad, Export_Ordnername))
  files.download('/content/Exportdaten_PPO.zip')

---
# Vergleichbare Charts
---

**BTCUSD**: [traidingview.com](https://de.tradingview.com/chart/ecCxGiMv/) *(login erforderlich)*

```
+-------------+------------+------------+-------------+-------------+
| TraidingView.com:                                                 |
+-------------+------------+------------+-------------+-------------+
|             | 2019-10-01 | 2019-09-30 | 2019-08-10  | 2019-08-07  |
+-------------+------------+------------+-------------+-------------+
| EMA26       |    9305.74 |    9384.68 |    10848.14 |    10630.29 |
| EMA12       |    8734.78 |    8810.38 |    11193.74 |    10886.97 |
| PPO         |      -6.14 |      -6.12 |        3.19 |        2.41 |
| SIGNAL      |      -4.56 |      -4.17 |        1.63 |        0.13 |
| PPO_History |      -1.57 |      -1.95 |        1.55 |        2.28 |
+-------------+------------+------------+-------------+-------------+

+-------------+------------+------------+-------------+-------------+
| CALCULATED:                                                       |
+-------------+------------+------------+-------------+-------------+
|             | 2019-10-01 | 2019-09-30 | 2019-08-10  | 2019-08-07  |
+-------------+------------+------------+-------------+-------------+
| EMA26       |  9327.3017 |  9446.6421 |  10651.5067 |  10673.4093 |
| EMA12       |  8751.5960 |  8842.4748 |  11163.8865 |  10785.1409 |
| PPO         |     -6.172 |    -6.3956 |      4.8104 |      1.0468 |
| SIGNAL      |    -4.4147 |    -4.0178 |      0.9102 |     -1.8162 |
| PPO_History |    -1.7576 |    -2.3778 |      3.9002 |      2.8631 |
+-------------+------------+------------+-------------+-------------+
```



**intc.csv (utf-8)** *--> eher ungeeignetes Beispiel*

 see [School.StockCharts.com](https://school.stockcharts.com/doku.php?id=technical_indicators:price_oscillators_ppo) 

```
time,open,close,high,low,volume
1271203140000,21.16,20.16,20.16,20.16,0.0
1271289540000,20.49,20.49,20.49,20.49,0.0
1271375940000,20.74,20.74,20.74,20.74,0.0
1271462340000,20.77,20.77,20.77,20.77,0.0
1271721540000,20.53,20.53,20.53,20.53,0.0
1271807940000,19.61,19.61,19.61,19.61,0.0
1271894340000,20.02,20.02,20.02,20.02,0.0
1271980740000,19.70,19.70,19.70,19.70,0.0
1272067140000,19.94,19.94,19.94,19.94,0.0
1272326340000,19.62,19.62,19.62,19.62,0.0
1272412740000,19.11,19.11,19.11,19.11,0.0
1272499140000,19.32,19.32,19.32,19.32,0.0
1272585540000,19.61,19.61,19.61,19.61,0.0
1272671940000,19.54,19.54,19.54,19.54,0.0
1272931140000,18.89,18.89,18.89,18.89,0.0
1273017540000,19.33,19.33,19.33,19.33,0.0
1273103940000,19.21,19.21,19.21,19.21,0.0
1273190340000,19.51,19.51,19.51,19.51,0.0
1273276740000,19.55,19.55,19.55,19.55,0.0
1273535940000,19.92,19.92,19.92,19.92,0.0
1273622340000,20.29,20.29,20.29,20.29,0.0
1273708740000,20.58,20.58,20.58,20.58,0.0
1273795140000,20.52,20.52,20.52,20.52,0.0
1273881540000,20.69,20.69,20.69,20.69,0.0
1274140740000,20.67,20.67,20.67,20.67,0.0
1274227140000,20.72,20.72,20.72,20.72,0.0
1274313540000,20.25,20.25,20.25,20.25,0.0
1274399940000,20.56,20.56,20.56,20.56,0.0
1274486340000,20.49,20.49,20.49,20.49,0.0
1274745540000,20.39,20.39,20.39,20.39,0.0

```

